In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.window import Window as W

spark = SparkSession.builder.appName('Puzzle').getOrCreate()

In [2]:
df = spark.read.csv('data/puzzle_041.csv', header=True, inferSchema=True)
df.show()

+-------+-----+--------+--------------+-------+
|deal_id|price|quantity|total_quantity| months|
+-------+-----+--------+--------------+-------+
|      2|  4.0|   10000|        310000|00:00.0|
|      2|  4.0|   10000|        280000|00:00.0|
|      2|  4.5|   10000|        310000|00:00.0|
|      2|  4.0|   10000|        310000|00:00.0|
|      2|  4.0|   15000|        280000|00:00.0|
|      2|  4.0|   15000|        310000|00:00.0|
+-------+-----+--------+--------------+-------+



In [35]:
df.select(
    '*',
    F.when((F.lag('price',1,0).over(W.orderBy(F.lit(1))) !=F.col('price')) | (F.lag('quantity',1,0).over(W.orderBy(F.lit(1))) !=F.col('quantity')),1
    ).otherwise(0).alias('ss'),
    F.row_number().over(W.orderBy(F.lit(1))).alias('Rn')
).withColumn(
        'Group',
        F.sum('ss').over(W.orderBy(F.col('Rn')))
).groupBy(['Group', 'deal_id', 'price','quantity']).agg(
    F.sum(F.col('total_quantity')).alias('Total_quantity'), 
    F.min(F.col('months')).alias('StartMonth'),
    F.max(F.col('months')).alias('EndMonth')
).drop('Group').show()

+-------+-----+--------+--------------+----------+--------+
|deal_id|price|quantity|Total_quantity|StartMonth|EndMonth|
+-------+-----+--------+--------------+----------+--------+
|      2|  4.0|   10000|        590000|   00:00.0| 00:00.0|
|      2|  4.5|   10000|        310000|   00:00.0| 00:00.0|
|      2|  4.0|   10000|        310000|   00:00.0| 00:00.0|
|      2|  4.0|   15000|        590000|   00:00.0| 00:00.0|
+-------+-----+--------+--------------+----------+--------+

